In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
print(os.listdir("/kaggle/input"))

In [ ]:
# Set the base path for the dataset
BASE_PATH = '/kaggle/input/llm-classification-finetuning'

In [ ]:
# Load the training data from a CSV file
df = pd.read_csv(f'{BASE_PATH}/train.csv') 

In [ ]:
# Import necessary libraries
import keras
import keras_hub
import numpy as np

In [ ]:
# Set the Keras backend to JAX
import os
os.environ["KERAS_BACKEND"] = "jax"  
# Import additional libraries
import keras_nlp
import keras
import tensorflow as tf

import numpy as np 
import pandas as pd
from tqdm import tqdm
import json

import matplotlib.pyplot as plt
import matplotlib as mpl
import plotly.express as px

In [ ]:
# Print the versions of the libraries being used
print("TensorFlow:", tf.__version__)
print("Keras:", keras.__version__)
print("KerasNLP:", keras_nlp.__version__)

In [ ]:
# Configuration class to hold hyperparameters and settings
class CFG:
    seed = 42  # Random seed
    preset = "deberta_v3_extra_small_en" # Name of pretrained models
    sequence_length = 512  # Input sequence length
    epochs = 3 # Training epochs
    batch_size = 16  # Batch size
    scheduler = 'cosine'  # Learning rate scheduler
    label2name = {0: 'winner_model_a', 1: 'winner_model_b', 2: 'winner_tie'}   # Mapping of labels to class names
    name2label = {v:k for k, v in label2name.items()}   # Reverse mapping for labels
    class_labels = list(label2name.keys())   # List of class labels
    class_names = list(label2name.values())   # List of class names

In [ ]:
# Set random seed for Keras
keras.utils.set_random_seed(CFG.seed)

In [ ]:
# Set mixed precision policy for training
keras.mixed_precision.set_global_policy("mixed_float16")

In [ ]:
# Load the training data again (redundant, can be removed)
BASE_PATH = '/kaggle/input/llm-classification-finetuning'

In [ ]:
df = pd.read_csv(f'{BASE_PATH}/train.csv') 
# Process the prompt and responses to extract the first element from the evaluated string

df["prompt"] = df.prompt.map(lambda x: eval(x)[0])
df["response_a"] = df.response_a.map(lambda x: eval(x.replace("null","''"))[0])
df["response_b"] = df.response_b.map(lambda x: eval(x.replace("null", "''"))[0])

# Determine the class name and label based on the maximum winner count

df["class_name"] = df[["winner_model_a", "winner_model_b" , "winner_tie"]].idxmax(axis=1)
df["class_label"] = df.class_name.map(CFG.name2label)

# Display the first few rows of the processed DataFrame
df.head()

In [ ]:
# Load the test data
test_df = pd.read_csv(f'{BASE_PATH}/test.csv')
# Process the test data similarly to the training data
test_df["prompt"] = test_df.prompt.map(lambda x: eval(x)[0])
test_df["response_a"] = test_df.response_a.map(lambda x: eval(x.replace("null","''"))[0])
test_df["response_b"] = test_df.response_b.map(lambda x: eval(x.replace("null", "''"))[0])

# Display the first few rows of the test DataFrame
test_df.head()


In [ ]:
# Function to create pairs of prompts and responses
def make_pairs(row):
    row["encode_fail"] = False
    try:
        prompt = row.prompt.encode("utf-8").decode("utf-8")
    except:
        prompt = ""
        row["encode_fail"] = True

    try:
        response_a = row.response_a.encode("utf-8").decode("utf-8")
    except:
        response_a = ""
        row["encode_fail"] = True

    try:
        response_b = row.response_b.encode("utf-8").decode("utf-8")
    except:
        response_b = ""
        row["encode_fail"] = True
        
    row['options'] = [f"Prompt: {prompt}\n\nResponse: {response_a}",  # Response from Model A
                      f"Prompt: {prompt}\n\nResponse: {response_b}"  # Response from Model B
                     ]
    return row

In [ ]:
df = df.apply(make_pairs, axis=1)  # Generate paired comparisons for training data
display(df.head(2)) 
test_df = test_df.apply(make_pairs, axis=1)  # Generate paired comparisons for test data
display(test_df.head(2))

In [ ]:
# Check distribution of encoding failures
df.encode_fail.value_counts(normalize=False)

In [ ]:
# Combine models into a single series
model_df = pd.concat([df.model_a, df.model_b])
# Count occurrences of each LLM
counts = model_df.value_counts().reset_index()
counts.columns = ['LLM', 'Count']
# Plot distribution of LLMs
fig = px.bar(counts, x='LLM', y='Count',
             title='Distribution of LLMs',
             color='Count', color_continuous_scale='viridis')

fig.update_layout(xaxis_tickangle=-45)  
fig.show()

In [ ]:
 # Count win occurrences
counts = df['class_name'].value_counts().reset_index()
counts.columns = ['Winner', 'Win Count']
# Plot winner distribution
fig = px.bar(counts, x='Winner', y='Win Count',
             title='Winner distribution for Train Data',
             labels={'Winner': 'Winner', 'Win Count': 'Win Count'},
             color='Winner', color_continuous_scale='viridis')

fig.update_layout(xaxis_title="Winner", yaxis_title="Win Count")

fig.show()

In [ ]:
from sklearn.model_selection import train_test_split  # Import package
# Stratified split to maintain label distribution
train_df, valid_df = train_test_split(df, test_size=0.2, stratify=df["class_label"])

In [ ]:
preprocessor = keras_nlp.models.DebertaV3Preprocessor.from_preset(
    preset=CFG.preset,
    sequence_length=CFG.sequence_length,)


In [ ]:
outs = preprocessor(df.options.iloc[0])  # Process options for the first row

# Display the shape of each processed output for debugging
for k, v in outs.items():
    print(k, ":", v.shape)

In [ ]:
def preprocess_fn(text, label=None):
    text = preprocessor(text)  # Tokenize and Preprocess text
    return (text, label) if label is not None else text

In [ ]:
# Optimize data loading
def build_dataset(texts, labels=None, batch_size=32,
                  cache=True, shuffle=1024):
    AUTO = tf.data.AUTOTUNE  # AUTOTUNE option
    slices = (texts,) if labels is None else (texts, keras.utils.to_categorical(labels, num_classes=3))  # Create slices
    ds = tf.data.Dataset.from_tensor_slices(slices)  # Create dataset from slices
    ds = ds.cache() if cache else ds  # Cache dataset if enabled
    ds = ds.map(preprocess_fn, num_parallel_calls=AUTO)  # Map preprocessing function
    opt = tf.data.Options()  # Create dataset options
    if shuffle: 
        ds = ds.shuffle(shuffle, seed=CFG.seed)  # Shuffle dataset if enabled
        opt.experimental_deterministic = False
    ds = ds.with_options(opt)  # Set dataset options
    ds = ds.batch(batch_size, drop_remainder=False)  # Batch dataset
    ds = ds.prefetch(AUTO)  # Prefetch next batch
    return ds 

In [ ]:
# Prepare datasets
train_texts = train_df.options.tolist()  # Extract training texts
train_labels = train_df.class_label.tolist()  # Extract training labels
train_ds = build_dataset(train_texts, train_labels,
                         batch_size=CFG.batch_size,
                         shuffle=True)
valid_texts = valid_df.options.tolist()  # Extract validation texts
valid_labels = valid_df.class_label.tolist()  # Extract validation labels
valid_ds = build_dataset(valid_texts, valid_labels,
                         batch_size=CFG.batch_size,
                         shuffle=False)

In [ ]:
import math

def get_lr_callback(batch_size=8, mode='cos', epochs=10, plot=False):
    lr_start, lr_max, lr_min = 1.0e-6, 0.6e-6 * batch_size, 1e-6 # Learning rate boundaries
    lr_ramp_ep, lr_sus_ep, lr_decay = 2, 0, 0.8
 # Initial ramp-up phase
    def lrfn(epoch):  
        if epoch < lr_ramp_ep: lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start
        elif epoch < lr_ramp_ep + lr_sus_ep: lr = lr_max
        elif mode == 'exp': lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min
        elif mode == 'step': lr = lr_max * lr_decay**((epoch - lr_ramp_ep - lr_sus_ep) // 2)
        elif mode == 'cos':
            decay_total_epochs, decay_epoch_index = epochs - lr_ramp_ep - lr_sus_ep + 3, epoch - lr_ramp_ep - lr_sus_ep
            phase = math.pi * decay_epoch_index / decay_total_epochs
            lr = (lr_max - lr_min) * 0.5 * (1 + math.cos(phase)) + lr_min
        return lr

    if plot:  # Plot lr curve if plot is True
        plt.figure(figsize=(10, 5))
        plt.plot(np.arange(epochs), [lrfn(epoch) for epoch in np.arange(epochs)], marker='o')
        plt.xlabel('epoch'); plt.ylabel('lr')
        plt.title('LR Scheduler')
        plt.show()

    return keras.callbacks.LearningRateScheduler(lrfn, verbose=False)  

In [ ]:
lr_cb = get_lr_callback(CFG.batch_size, plot=True)

In [ ]:
ckpt_cb = keras.callbacks.ModelCheckpoint(f'best_model.weights.h5',
                                          monitor='val_log_loss',
                                          save_best_only=True,
                                          save_weights_only=True,
                                          mode='min')

In [ ]:
log_loss = keras.metrics.CategoricalCrossentropy(name="log_loss")

In [ ]:
inputs = {
    "token_ids": keras.Input(shape=(2, None), dtype=tf.int32, name="token_ids"),
    "padding_mask": keras.Input(shape=(2, None), dtype=tf.int32, name="padding_mask"),
}
backbone = keras_nlp.models.DebertaV3Backbone.from_preset(
    CFG.preset,
)


response_a = {k: v[:, 0, :] for k, v in inputs.items()}
embed_a = backbone(response_a)


response_b = {k: v[:, 1, :] for k, v in inputs.items()}
embed_b = backbone(response_b)


embeds = keras.layers.Concatenate(axis=-1)([embed_a, embed_b])
embeds = keras.layers.GlobalAveragePooling1D()(embeds)
outputs = keras.layers.Dense(3, activation="softmax", name="classifier")(embeds)
model = keras.Model(inputs, outputs)


model.compile(
    optimizer=keras.optimizers.Adam(5e-6),
    loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.02),
    metrics=[
        log_loss,
        keras.metrics.CategoricalAccuracy(name="accuracy"),
    ],
)

In [ ]:
model.summary()

In [ ]:
test_texts = test_df.options.tolist()
test_ds = build_dataset(test_texts,
                         batch_size=min(len(test_df), CFG.batch_size),
                         shuffle=False)

In [ ]:
test_preds = model.predict(test_ds, verbose=1)

In [ ]:
sub_df = test_df[["id"]].copy()
sub_df[CFG.class_names] = test_preds.tolist()
sub_df.to_csv("submission.csv", index=False)
sub_df.head()